In [2]:
from datetime import datetime
import mysql.connector
import streamlit as st
import mysql.connector
from dotenv import load_dotenv
import os
import requests
from bs4 import BeautifulSoup
import openai
from gtts import gTTS
from io import BytesIO

In [7]:
NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")
load_dotenv()
db_config = {
    'host': os.getenv('DB_HOST'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'database': os.getenv('DB_NAME')
}
openai.api_key = os.getenv("OPEN_AI_API")

In [8]:
## 실제로 찍으면안됨

if not NAVER_CLIENT_ID or not NAVER_CLIENT_SECRET:
    raise ValueError("NAVER_CLIENT_ID and NAVER_CLIENT_SECRET 이 설정되지 않았습니다")

In [9]:

class NaverNews:
    def __init__(self, id: int, title: str, originallink: str, link: str, description: str, pubDate: str, created_at: datetime=None):
        self.__id = id
        self.__title = title
        self.__originallink = originallink
        self.__link = link
        self.__description = description
        self.__pubDate = pubDate
        self.__created_at = created_at

    @property
    def id(self):
        return self.__id

    @property
    def title(self):
        return self.__title

    # __title속성에 대한 setter 메소드
    # @title.setter
    # def title(self, value):
    #     self.__title = value

    @property
    def originallink(self):
        return self.__originallink

    @property
    def link(self):
        return self.__link

    @property
    def description(self):
        return self.__description

    @property
    def pubDate(self):
        return self.__pubDate

    @property
    def created_at(self):
        return self.__created_at

    def __repr__(self):
        return f'NaverNews({self.__id}, {self.__title}, {self.__originallink}, {self.__link}, {self.__description}, {self.__pubDate}, {self.__created_at})'

In [10]:


url = 'https://openapi.naver.com/v1/search/news.json'

headers = {
    "X-Naver-Client-Id": NAVER_CLIENT_ID,
    "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
}
user_input = input('교통사고입력')
# 사용자입력값 (query string)
params = {
    'query': f'{user_input} 교통사고',
    'display': 3, # 10 ~ 100
    'start': 1,
    'sort': 'sim', # sim | date
}

# 요청
response = requests.get(url, headers=headers, params=params)

naver_news_list: list[NaverNews] = []

# 결과출력
if response.status_code == 200:
    # print(response.text) # json문자열 확인
    data = response.json() # json형식의 데이터를 dict 변환
    items = data['items']
    for item in items:
        naver_news_list.append(NaverNews(None, **item))
    print(naver_news_list)
else:
    print("Error Code:" + response.status_code)

[NaverNews(None, &quot;어르신 운전중입니다&quot;…<b>서울시</b>, 차량 4600대에 표지판 부착, https://www.newsis.com/view/NISX20250427_0003155174, https://n.news.naver.com/mnews/article/003/0013209082?sid=102, 지난해 <b>서울시</b>의 65세 이상 고령 운전자 <b>교통사고</b> 건수는 전년 대비 5.6% 증가한 것으로 나타났다. 특히, 사망자 수는 57.1% 급증했다. 실제 '어르신 운전중' 표지 부착 효과도 확인되고 있다. 지난해 한국<b>교통</b>안전공단... , Mon, 28 Apr 2025 06:00:00 +0900, None), NaverNews(None, <b>서울시</b>, 고령운전자에 '어르신 운전중' 표지 4천600매 배부, https://www.yna.co.kr/view/AKR20250425170700004?input=1195m, https://n.news.naver.com/mnews/article/001/0015355265?sid=102, 작년 고령운전 <b>교통사고</b> 5.6% 늘고 사망자 57% 급증 <b>서울시</b> 자치경찰위원회(자경위)는 65세 이상 고령자가 운전하는 차량에 부착할 수 있는 '어르신 운전중' 표지를 제작해 배부한다고 28일 밝혔다. 가정의 달을 맞아 표지... , Mon, 28 Apr 2025 06:00:00 +0900, None), NaverNews(None, <b>교통사고</b> 사망 주는데, 고령운전 사망 3년째 증가, https://www.donga.com/news/Society/article/all/20250428/131503233/2, https://n.news.naver.com/mnews/article/020/0003631379?sid=102, 경찰청에 따르면 지난해 가해자가 고령 운전자인 <b>교통사고</b>의 사망자는 761명으로, 2022년(735명), 2023년(

In [11]:
try:
    with mysql.connector.connect(**db_config) as conn:
        with conn.cursor() as cursor:
            # naver_news객체를 한번에 한건씩 insert처리
            for naver_news in naver_news_list:
                cursor.execute('''
                    insert into
                        naver_news (title, originallink, link, description, pub_date)
                    values (%s, %s, %s, %s, %s)
                ''', (naver_news.title, naver_news.originallink, naver_news.link, naver_news.description, naver_news.pubDate)) # sql, params
            # 트랜잭션 처리를 위해 모든 dml 작업후 마지막에 처리
            conn.commit()
except mysql.connector.Error as e:
    print('DB 오류:', e)

In [12]:
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/89.0.4389.114 Safari/537.36"
    )
} ## 유저인척하기


# ── 3) 유틸: HTML 전처리 ─────────────────────────
def clean_html(soup: BeautifulSoup):
    for tag in soup.select("script, style, aside, .ad, .related-article"):
        tag.decompose()


# ── 4) 전체 텍스트 가져오기 ─────────────────────
def get_html_text(url: str) -> str:
    try:
        res = requests.get(url, headers=HEADERS, timeout=10)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "html.parser")
        clean_html(soup)
        return soup.get_text(separator="\n")
    except Exception:
        return ""


# ── 5) 요약하기 (공백 정리 추가) ─────────────────
def summarize_text(text: str) -> str:
    cleaned_text = "\n".join(
        line.strip() for line in text.splitlines() if line.strip()
    )

    prompt = f"""
    다음 웹페이지 텍스트를 읽고 중요 기사 내용만 3줄로 요약해줘.
    메뉴, 광고, 댓글, 저작권 문구는 무시해줘.

    {cleaned_text}
    """
    resp = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5,
    )
    return resp.choices[0].message.content.strip()



In [13]:
st.title("📰 뉴스 요약 & TTS 변환")

st.caption("네이버 뉴스 원문 링크를 요약하고 음성으로 변환합니다.")

if st.button("📝 뉴스 요약 시작"):
    try:
        conn = mysql.connector.connect(**db_config)
        cur = conn.cursor()
        cur.execute("SELECT originallink FROM naver_news")
        urls = [r[0] for r in cur.fetchall()]
        cur.close()
        conn.close()
    except Exception as e:
        st.error(f"DB 오류: {e}")
        urls = []

    for idx, url in enumerate(urls, start=1):
        with st.container():
            st.markdown(f"### 🔗 뉴스 #{idx}")
            st.markdown(f"[{url}]({url})")

            article = get_html_text(url)
            if not article:
                st.error("❗ 전체 텍스트 가져오기 실패")
                continue

            summary = summarize_text(article)

            if "무단 전재" in summary or "재배포 금지" in summary:
                st.warning("🚫 해당 기사는 TTS가 불가합니다. (무단 전재 문구 포함)")
                continue

            st.success("✨ 요약 결과")
            st.write(summary)

            tts = gTTS(text=summary, lang="ko")
            buf = BytesIO()
            tts.write_to_fp(buf)

            st.audio(buf.getvalue(), format="audio/mp3")

        st.markdown("---")  # 구분선 추가

2025-04-28 10:20:10.847 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-28 10:20:10.875 
  command:

    streamlit run C:\Users\Playdata2\miniconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-28 10:20:10.876 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-28 10:20:10.877 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-28 10:20:10.877 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-28 10:20:10.878 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-28 10:20:10.880 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-28 10:20:10.880 Thread 'MainThread